In [ ]:
%%bash
# Go to folder containing the source code
cd /app/mialsuperresolutiontoolkit/
# Install the pymialsrtk package inside the python/conda environment
python setup.py install --force

In [ ]:
import os
import json
import shutil

# Imports from nipype
from nipype.interfaces.io import BIDSDataGrabber,DataGrabber
from nipype.pipeline import Node, Workflow

# Import the implemented interface from pymialsrtk
import pymialsrtk.interfaces.preprocess as preprocess
import pymialsrtk.interfaces.reconstruction as reconstruction

# Cpoy result files
from shutil import copyfile
import glob

In [3]:
# Set different variables (defined in cell 2) such that we do not have to rerun cell 2
bids_dir = os.path.join('/fetaldata')

if 1:
    session = None
    subject = 'sub-VD32'
    stacksOrder = [6,4,2,5,0,1]

else:
    subject = 'sub-ctrl0022'
    session = 'ses-20160517084936'
    stacksOrder = [4,2,7,5,9,1,3,6]


In [ ]:
from nipype import config, logging

In [ ]:

import os

def run(command, env={}, cwd=os.getcwd()):
    import subprocess
    merged_env = os.environ
    merged_env.update(env)
    process = subprocess.run(command, shell=True,
                             env=merged_env, cwd=cwd, capture_output=True)
    return process


In [6]:
# 
# Copy result files to BIDS
def copy_results_to_bids(p_bids_dir, nipype_dir, p_wf_base_dir, p_stacksOrder, subject, session=None):

    create_json_path = os.path.join(p_bids_dir, "code", "create_scan_preproc_json.sh")
    if session is None:
        t2w_dir = os.path.join(p_bids_dir, subject, "anat")
        mask_dir = os.path.join(p_bids_dir, 'derivatives', 'manual_masks', subject, "anat")
        
        anat_dir = os.path.join(p_wf_base_dir, "anat")
        if not os.path.exists(anat_dir):
            os.makedirs(anat_dir)

        xfm_dir = os.path.join(p_wf_base_dir, "xfm")
        if not os.path.exists(xfm_dir):
            os.makedirs(xfm_dir)
    else:        
        t2w_dir = os.path.join(bids_dir, subject, session, "anat")
        mask_dir = os.path.join(bids_dir, 'derivatives', 'manual_masks', subject, session, "anat")

        anat_dir = os.path.join(p_wf_base_dir, "anat")
        if not os.path.exists(anat_dir):
            os.makedirs(anat_dir)

        xfm_dir = os.path.join(p_wf_base_dir, "xfm")
        if not os.path.exists(xfm_dir):
            os.makedirs(xfm_dir)

    uni_bcorr_histnorm_dir = os.path.join(nipype_dir, "srtkIntensityStandardization02")
    transform_nV_dir = os.path.join(nipype_dir, "srtkImageReconstruction")

  
    sources = ""
    sources_lst = []
    for num_stack in p_stacksOrder:
        
        
        preproc_files = glob.glob(os.path.join(uni_bcorr_histnorm_dir, ''.join(["*run-",str(num_stack), "_*T2w_uni_bcorr_histnorm.nii.gz"])))[0]
        dst_T2w = os.path.join(anat_dir, os.path.basename(preproc_files).replace("_uni_bcorr_histnorm", "_preproc"))
        copyfile(preproc_files, dst_T2w)

        transform_files = glob.glob(os.path.join(transform_nV_dir, ''.join(["*run-",str(num_stack), "_*T2w_nlm_uni_bcorr_histnorm_transform_*V.txt"])))[0]
        filename = os.path.basename(transform_files)
        dst_transf = os.path.join(xfm_dir, filename.replace(filename[filename.find("_transform"):], "_from-orig_to-SDI_mode-image_xfm.txt"))
        copyfile(transform_files, dst_transf)

        
        init_mask = glob.glob(os.path.join(mask_dir, ''.join(["*run-", str(num_stack), "*mask.nii.gz" ])))[0]
        copyfile(init_mask, os.path.join(anat_dir, os.path.basename(init_mask)))

    #     print(os.path.join(t2w_dir, ''.join(["*_run-", str(num_stack), "_T2w.nii.gz" ])))
    #     init_T2w = glob.glob(os.path.join(t2w_dir, ''.join(["*_run-", str(num_stack), "_T2w.nii.gz" ])))[0]
    #     print(' '.join(['sh', create_json_path, init_T2w]))
    #     run(' '.join(['sh', create_json_path, init_T2w]))


        sources += os.path.join(anat_dir, filename.replace("_uni_bcorr_histnorm", "_preproc")) + ", "
        sources += os.path.join(xfm_dir, filename.replace(filename[filename.find("_transform"):], "_from-orig_to-SDI_mode-image_xfm.txt")) + ", "
        sources += os.path.join(anat_dir, os.path.basename(init_mask)) + ", "

        sources_lst.append(os.path.join(anat_dir, filename.replace("_uni_bcorr_histnorm", "_preproc")))
        sources_lst.append(os.path.join(xfm_dir, filename.replace(filename[filename.find("_transform"):], "_from-orig_to-SDI_mode-image_xfm.txt")))
        sources_lst.append(os.path.join(anat_dir, os.path.basename(init_mask)))




    sdi_dir = os.path.join(nipype_dir, "srtkImageReconstruction")
    srtv_dir = os.path.join(nipype_dir, "srtkN4BiasFieldCorrection")
    srtv_masked_dir = os.path.join(nipype_dir, "srtkMaskImage02")

    print("Copy final outputs to ${ANAT_DIR}")

    # SDI 
    sdi_files = glob.glob(os.path.join(sdi_dir, ''.join(["SDI_*.nii.gz"])))[0]
    filename = os.path.basename(sdi_files)
    dst_sdi = '_'.join([filename.replace('SDI_', '').split(''.join(['_',str(len(p_stacksOrder)), "V_"]))[0], 'rec-SDI', 'T2w.nii.gz'])
    copyfile(sdi_files, os.path.join(anat_dir, dst_sdi))
    print("")
    print(sdi_files)
    print("to")
    print(os.path.join(anat_dir, dst_sdi))
    print("")


    # SRTV 
    srtv_files = glob.glob(os.path.join(srtv_dir, ''.join(["SRTV_*_gbcorr.nii.gz"])))[0]
    filename = os.path.basename(srtv_files)
    dst_srtv = '_'.join([filename.replace('SRTV_', '').split(''.join(['_',str(len(p_stacksOrder)), "V_"]))[0], 'rec-SR', 'T2w.nii.gz'])
    copyfile(srtv_files, os.path.join(anat_dir, dst_srtv))
    print("")
    print(srtv_files)
    print("to")
    print(os.path.join(anat_dir, dst_srtv))
    print("")

    # to do - masked SRTV/SDI ?  
    srtv_masked_files = glob.glob(os.path.join(srtv_masked_dir, ''.join(["SRTV_*.nii.gz"])))[0]
    filename = os.path.basename(srtv_masked_files)
    dst_srtv = '_'.join([filename.replace('SRTV_', '').split(''.join(['_',str(len(p_stacksOrder)), "V_"]))[0], 'rec-SRmasked', 'T2w.nii.gz'])
    copyfile(srtv_masked_files, os.path.join(anat_dir, dst_srtv))
    print("")
    print(srtv_files)
    print("to")
    print(os.path.join(anat_dir, dst_srtv))
    print("")



    output_dict = {}

    output_dict["Description"] = "Isotropic high-resolution image reconstructed using the Total-Variation Super-Resolution algorithm provided by MIALSRTK"
    # output_dict["Sources"] = sources
    output_dict["Sources"] = sources_lst
    output_dict["CustomMetaData"] = {}
    output_dict["CustomMetaData"]["Number of scans used"] = str(len(p_stacksOrder))
    output_dict["CustomMetaData"]["TV regularization weight lambda"] = "4"
    output_dict["CustomMetaData"]["Optimization time step"] = "3"
    output_dict["CustomMetaData"]["Primal/dual loops"] = "2"
    output_dict["CustomMetaData"]["Number of pipeline iterations"] = "1"

    output_json = os.path.join(anat_dir, ''.join([subject, '_rec-SR.json']))
    with open(output_json, 'w+', encoding='utf8') as outfile:
        json.dump(output_dict, outfile, indent=4)

In [10]:
## Node linkage
def create_workflow(bids_dir, subject, p_stacksOrder, session=None):
#     wf_base_dir = os.path.join("{}".format(output_dir),"superres-mri","sub-{}".format(subject),"nipype")
    output_dir = os.path.join("{}".format(bids_dir),"derivatives","superres-mri")
    
    if session is None:
        wf_base_dir = os.path.join(output_dir, subject)
    else:
        wf_base_dir = os.path.join(output_dir, subject, session)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    print("Ouput directory: {}".format(wf_base_dir))

    wf = Workflow(name="srr_nipype",base_dir=wf_base_dir)
    srr_nipype_dir = os.path.join(wf.base_dir, wf.name )
    
    
    # Initialization
    if os.path.isfile(os.path.join(output_dir,"pypeline_"+subject+".log")):
        os.unlink(os.path.join(output_dir,"pypeline_"+subject+".log"))
#         open(os.path.join(output_dir,"pypeline.log"), 'a').close()
        

    config.update_config({'logging': {'log_directory': os.path.join(output_dir), 'log_to_file': True},
                          'execution': {
                              'remove_unnecessary_outputs': False,
                              'stop_on_first_crash': True,
                              'stop_on_first_rerun': False,
                              'crashfile_format': "txt",
                              'write_provenance' : False,},
                          'monitoring': { 'enabled': True }
                        })
    
    logging.update_logging(config)
    iflogger = logging.getLogger('nipype.interface')

    iflogger.info("**** Processing ****")

    
    dg = Node(interface=DataGrabber(outfields = ['T2ws', 'masks']), name='data_grabber')
    
    dg.inputs.base_directory = bids_dir
    dg.inputs.template = '*'
    dg.inputs.raise_on_empty = False
    dg.inputs.sort_filelist=True
    
    dg.inputs.field_template = dict(T2ws=os.path.join(subject, 'anat', subject+'*_run-*_T2w.nii.gz'),
                                   masks=os.path.join('derivatives','manual_masks', subject, 'anat', subject+'*_run-*_*mask.nii.gz'))
    if not (session is None):
        dg.inputs.field_template = dict(T2ws=os.path.join( subject, session, 'anat', '_'.join([subject, session, '*run-*', '*T2w.nii.gz'])),
                                        masks=os.path.join('derivatives','manual_masks', subject, session, 'anat','_'.join([subject, session, '*run-*', '*mask.nii.gz'])))
    
    
        
    nlmDenoise = Node(interface=preprocess.MultipleBtkNLMDenoising(), name='nlmDenoise')
    nlmDenoise.inputs.bids_dir = bids_dir
    nlmDenoise.inputs.stacksOrder = p_stacksOrder
    nlmDenoise.inputs.weight = 1
    
    # Sans le mask le premier correct slice intensity...
    srtkCorrectSliceIntensity01_nlm = Node(interface=preprocess.MultipleMialsrtkCorrectSliceIntensity(), name='srtkCorrectSliceIntensity01_nlm')
    srtkCorrectSliceIntensity01_nlm.inputs.bids_dir = bids_dir
    srtkCorrectSliceIntensity01_nlm.inputs.stacksOrder = p_stacksOrder
    srtkCorrectSliceIntensity01_nlm.inputs.out_postfix = '_uni'

    srtkCorrectSliceIntensity01 = Node(interface=preprocess.MultipleMialsrtkCorrectSliceIntensity(), name='srtkCorrectSliceIntensity01')
    srtkCorrectSliceIntensity01.inputs.bids_dir = bids_dir
    srtkCorrectSliceIntensity01.inputs.stacksOrder = p_stacksOrder
    srtkCorrectSliceIntensity01.inputs.out_postfix = '_uni'

    
    
    srtkSliceBySliceN4BiasFieldCorrection = Node(interface=preprocess.MultipleMialsrtkSliceBySliceN4BiasFieldCorrection(), name='srtkSliceBySliceN4BiasFieldCorrection')
    srtkSliceBySliceN4BiasFieldCorrection.inputs.bids_dir = bids_dir
    srtkSliceBySliceN4BiasFieldCorrection.inputs.stacksOrder = p_stacksOrder
    
    srtkSliceBySliceCorrectBiasField = Node(interface=preprocess.MultipleMialsrtkSliceBySliceCorrectBiasField(), name='srtkSliceBySliceCorrectBiasField')
    srtkSliceBySliceCorrectBiasField.inputs.bids_dir = bids_dir
    srtkSliceBySliceCorrectBiasField.inputs.stacksOrder = p_stacksOrder
    
    
    
    srtkCorrectSliceIntensity02_nlm = Node(interface=preprocess.MultipleMialsrtkCorrectSliceIntensity(), name='srtkCorrectSliceIntensity02_nlm')
    srtkCorrectSliceIntensity02_nlm.inputs.bids_dir = bids_dir
    srtkCorrectSliceIntensity02_nlm.inputs.stacksOrder = p_stacksOrder

    srtkCorrectSliceIntensity02 = Node(interface=preprocess.MultipleMialsrtkCorrectSliceIntensity(), name='srtkCorrectSliceIntensity02')
    srtkCorrectSliceIntensity02.inputs.bids_dir = bids_dir
    srtkCorrectSliceIntensity02.inputs.stacksOrder = p_stacksOrder
    
    
    srtkIntensityStandardization01 = Node(interface=preprocess.MialsrtkIntensityStandardization(), name='srtkIntensityStandardization01')
    srtkIntensityStandardization01.inputs.bids_dir = bids_dir
    
    
    srtkIntensityStandardization01_nlm = Node(interface=preprocess.MialsrtkIntensityStandardization(), name='srtkIntensityStandardization01_nlm')
    srtkIntensityStandardization01_nlm.inputs.bids_dir = bids_dir
    
    
    srtkHistogramNormalization = Node(interface=preprocess.MialsrtkHistogramNormalization(), name='srtkHistogramNormalization')
    srtkHistogramNormalization.inputs.bids_dir = bids_dir
    srtkHistogramNormalization.inputs.stacksOrder = p_stacksOrder
    
    srtkHistogramNormalization_nlm = Node(interface=preprocess.MialsrtkHistogramNormalization(), name='srtkHistogramNormalization_nlm')  
    srtkHistogramNormalization_nlm.inputs.bids_dir = bids_dir
    srtkHistogramNormalization_nlm.inputs.stacksOrder = p_stacksOrder
    
    
    
    srtkIntensityStandardization02 = Node(interface=preprocess.MialsrtkIntensityStandardization(), name='srtkIntensityStandardization02')
    srtkIntensityStandardization02.inputs.bids_dir = bids_dir
    
    
    srtkIntensityStandardization02_nlm = Node(interface=preprocess.MialsrtkIntensityStandardization(), name='srtkIntensityStandardization02_nlm')
    srtkIntensityStandardization02_nlm.inputs.bids_dir = bids_dir
    
    
    srtkMaskImage01 = Node(interface=preprocess.MultipleMialsrtkMaskImage(), name='srtkMaskImage01')
    srtkMaskImage01.inputs.bids_dir = bids_dir
    srtkMaskImage01.inputs.stacksOrder = p_stacksOrder


    srtkImageReconstruction = Node(interface=reconstruction.MialsrtkImageReconstruction(), name='srtkImageReconstruction')  
    srtkImageReconstruction.inputs.bids_dir = bids_dir
    srtkImageReconstruction.inputs.in_roi = "mask"
    srtkImageReconstruction.inputs.stacksOrder = p_stacksOrder 
    srtkImageReconstruction.inputs.in_iter = 1  
    
    sub_ses = subject
    if session != None:
        sub_ses = ''.join([sub_ses, '_', session])
    srtkImageReconstruction.inputs.sub_ses = sub_ses

    
    srtkTVSuperResolution = Node(interface=reconstruction.MialsrtkTVSuperResolution(), name='srtkTVSuperResolution')  
    srtkTVSuperResolution.inputs.bids_dir = bids_dir
    srtkTVSuperResolution.inputs.deblurring = True
    srtkTVSuperResolution.inputs.stacksOrder = p_stacksOrder
    srtkTVSuperResolution.inputs.sub_ses = sub_ses
    
    

    srtkRefineHRMaskByIntersection = Node(interface=preprocess.MialsrtkRefineHRMaskByIntersection(), name='srtkRefineHRMaskByIntersection')
    srtkRefineHRMaskByIntersection.inputs.bids_dir = bids_dir
    srtkRefineHRMaskByIntersection.inputs.stacksOrder = p_stacksOrder
    
    srtkN4BiasFieldCorrection = Node(interface=preprocess.MialsrtkN4BiasFieldCorrection(), name='srtkN4BiasFieldCorrection')
    srtkN4BiasFieldCorrection.inputs.bids_dir = bids_dir
    
    
    srtkMaskImage02 = Node(interface=preprocess.MialsrtkMaskImage(), name='srtkMaskImage02')
    srtkMaskImage02.inputs.bids_dir = bids_dir
    
    #
    ## Nodes ready - Linking now
    
    wf.connect(dg, "T2ws", nlmDenoise, "input_images")
    wf.connect(dg, "masks", nlmDenoise, "input_masks")
    
    wf.connect(nlmDenoise, "output_images", srtkCorrectSliceIntensity01_nlm, "input_images")
    wf.connect(dg, "masks", srtkCorrectSliceIntensity01_nlm, "input_masks")
    
    wf.connect(dg, "T2ws", srtkCorrectSliceIntensity01, "input_images")
    wf.connect(dg, "masks", srtkCorrectSliceIntensity01, "input_masks")
    
    wf.connect(srtkCorrectSliceIntensity01_nlm, "output_images", srtkSliceBySliceN4BiasFieldCorrection, "input_images")
    wf.connect(dg, "masks", srtkSliceBySliceN4BiasFieldCorrection, "input_masks")
    
    wf.connect(srtkCorrectSliceIntensity01, "output_images", srtkSliceBySliceCorrectBiasField, "input_images")
    wf.connect(srtkSliceBySliceN4BiasFieldCorrection, "output_fields", srtkSliceBySliceCorrectBiasField, "input_fields")
    wf.connect(dg, "masks", srtkSliceBySliceCorrectBiasField, "input_masks")
    
    wf.connect(srtkSliceBySliceCorrectBiasField, "output_images", srtkCorrectSliceIntensity02, "input_images")
    wf.connect(dg, "masks", srtkCorrectSliceIntensity02, "input_masks")
    
    wf.connect(srtkSliceBySliceN4BiasFieldCorrection, "output_images", srtkCorrectSliceIntensity02_nlm, "input_images")
    wf.connect(dg, "masks", srtkCorrectSliceIntensity02_nlm, "input_masks")
    
    wf.connect(srtkCorrectSliceIntensity02, "output_images", srtkIntensityStandardization01, "input_images")
    
    wf.connect(srtkCorrectSliceIntensity02_nlm, "output_images", srtkIntensityStandardization01_nlm, "input_images")
    
    wf.connect(srtkIntensityStandardization01, "output_images", srtkHistogramNormalization, "input_images")
    wf.connect(dg, "masks", srtkHistogramNormalization, "input_masks")
    
    wf.connect(srtkIntensityStandardization01_nlm, "output_images", srtkHistogramNormalization_nlm, "input_images")
    wf.connect(dg, "masks", srtkHistogramNormalization_nlm, "input_masks")
    
    wf.connect(srtkHistogramNormalization, "output_images", srtkIntensityStandardization02, "input_images")
    
    wf.connect(srtkHistogramNormalization_nlm, "output_images", srtkIntensityStandardization02_nlm, "input_images")
    
    
    wf.connect(srtkIntensityStandardization02_nlm, "output_images", srtkMaskImage01, "input_images")
    wf.connect(dg, "masks", srtkMaskImage01, "input_masks")
    
    wf.connect(srtkMaskImage01, "output_images", srtkImageReconstruction, "input_images")
    wf.connect(dg, "masks", srtkImageReconstruction, "input_masks")
    
    
    wf.connect(srtkIntensityStandardization02, "output_images", srtkTVSuperResolution, "input_images")
    wf.connect(srtkImageReconstruction, "output_transforms", srtkTVSuperResolution, "input_transforms")
    wf.connect(dg, "masks", srtkTVSuperResolution, "input_masks")
    wf.connect(srtkImageReconstruction, "output_sdi", srtkTVSuperResolution, "input_sdi")
    
    
    wf.connect(srtkIntensityStandardization02, "output_images", srtkRefineHRMaskByIntersection, "input_images")
    wf.connect(dg, "masks", srtkRefineHRMaskByIntersection, "input_masks")
    wf.connect(srtkImageReconstruction, "output_transforms", srtkRefineHRMaskByIntersection, "input_transforms")
    wf.connect(srtkTVSuperResolution, "output_sr", srtkRefineHRMaskByIntersection, "input_sr")
    
    wf.connect(srtkTVSuperResolution, "output_sr", srtkN4BiasFieldCorrection, "input_image")
    wf.connect(srtkRefineHRMaskByIntersection, "output_SRmask", srtkN4BiasFieldCorrection, "input_mask")
    
    wf.connect(srtkTVSuperResolution, "output_sr", srtkMaskImage02, "in_file")
    wf.connect(srtkRefineHRMaskByIntersection, "output_SRmask", srtkMaskImage02, "in_mask")
    
    
        
    return wf

In [ ]:
# m_wf = create_workflow(bids_dir, subject, p_stacksOrder=stacsksOrder)
m_wf = create_workflow(bids_dir, subject, p_stacksOrder=stacksOrder, session = session)
m_wf.write_graph()
aa = m_wf.run()


copy_results_to_bids(p_bids_dir= bids_dir, 
                     nipype_dir=os.path.join(m_wf.base_dir, m_wf.name), 
                     p_wf_base_dir=m_wf.base_dir, 
                     p_stacksOrder=stacksOrder, 
                     subject=subject, 
                     session=session)

In [11]:
import pandas as pd

bids_dir = os.path.join('/fetaldata')

hh = pd.read_csv(os.path.join('/fetaldata', 'code', 'batch_list.csv'))
hh["Session"] = hh["Session"].fillna('')
for ind, row in hh.iterrows():
    sub = row["Subjects"]
    ses = None
    if row["Session"] != '':
        ses = row["Session"]
        
    
    stacksOrder = [int(n) for n in row["StacksOrder"].split(',')]
    print(stacksOrder)
    
    # m_wf = create_workflow(bids_dir, subject, p_stacksOrder=stacsksOrder)
    m_wf = create_workflow(bids_dir, sub, p_stacksOrder=stacksOrder, session = ses)
    m_wf.write_graph()
    aa = m_wf.run()


    copy_results_to_bids(p_bids_dir= bids_dir, 
                         nipype_dir=os.path.join(m_wf.base_dir, m_wf.name), 
                         p_wf_base_dir=m_wf.base_dir, 
                         p_stacksOrder=stacksOrder, 
                         subject=sub, 
                         session=ses)

[6, 4, 2, 5, 0, 1]
Ouput directory: /fetaldata/derivatives/superres-mri/sub-VD32
200526-06:51:59,415 nipype.interface INFO:
	 **** Processing ****
200526-06:51:59,617 nipype.workflow INFO:
	 Generated workflow graph: /fetaldata/derivatives/superres-mri/sub-VD32/srr_nipype/graph.png (graph2use=hierarchical, simple_form=True).
200526-06:51:59,622 nipype.workflow INFO:
	 Workflow srr_nipype settings: ['check', 'execution', 'logging', 'monitoring']
200526-06:51:59,633 nipype.workflow INFO:
	 Running serially.
200526-06:51:59,634 nipype.workflow INFO:
	 [Node] Setting-up "srr_nipype.data_grabber" in "/fetaldata/derivatives/superres-mri/sub-VD32/srr_nipype/data_grabber".
200526-06:51:59,635 nipype.workflow INFO:
	 [Node] Outdated cache found for "srr_nipype.data_grabber".
200526-06:51:59,637 nipype.workflow INFO:
	 [Node] Running "data_grabber" ("nipype.interfaces.io.DataGrabber")
200526-06:51:59,641 nipype.workflow INFO:
	 [Node] Finished "srr_nipype.data_grabber".
200526-06:51:59,642 nipyp

TypeError: '_Undefined' object is not iterable

In [ ]:
print(a)

In [9]:
sub = 'sub-HK01'
ses = 'ses-01'
stacksOrder= [19,20,21]


m_wf = create_workflow(bids_dir, sub, p_stacksOrder=stacksOrder, session = ses)
m_wf.write_graph()
aa = m_wf.run()


copy_results_to_bids(p_bids_dir= bids_dir, 
                     nipype_dir=os.path.join(m_wf.base_dir, m_wf.name), 
                     p_wf_base_dir=m_wf.base_dir, 
                     p_stacksOrder=stacksOrder, 
                     subject=sub, 
                     session=ses)

Ouput directory: /fetaldata/derivatives/superres-mri/sub-HK01/ses-01
200526-06:49:56,597 nipype.interface INFO:
	 **** Processing ****
200526-06:49:56,810 nipype.workflow INFO:
	 Generated workflow graph: /fetaldata/derivatives/superres-mri/sub-HK01/ses-01/srr_nipype/graph.png (graph2use=hierarchical, simple_form=True).
200526-06:49:56,815 nipype.workflow INFO:
	 Workflow srr_nipype settings: ['check', 'execution', 'logging', 'monitoring']
200526-06:49:56,827 nipype.workflow INFO:
	 Running serially.
200526-06:49:56,828 nipype.workflow INFO:
	 [Node] Setting-up "srr_nipype.data_grabber" in "/fetaldata/derivatives/superres-mri/sub-HK01/ses-01/srr_nipype/data_grabber".
200526-06:49:56,830 nipype.workflow INFO:
	 [Node] Running "data_grabber" ("nipype.interfaces.io.DataGrabber")
200526-06:49:56,835 nipype.workflow INFO:
	 [Node] Finished "srr_nipype.data_grabber".
200526-06:49:56,835 nipype.workflow INFO:
	 [Node] Setting-up "srr_nipype.srtkCorrectSliceIntensity01" in "/fetaldata/derivati

/opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/nipype/interfaces/io.py:1204: UserWarning: Output key: T2ws Template: /fetaldata/sub-HK01/ses-01/anat/sub-HK01_ses-01_*run-*_*T2w.nii.gz returned no files
  warn(msg)
/opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/nipype/interfaces/io.py:1204: UserWarning: Output key: masks Template: /fetaldata/derivatives/manual_masks/sub-HK01/ses-01/anat/sub-HK01_ses-01_*run-*_*mask.nii.gz returned no files
  warn(msg)


TraitError: Each element of the 'input_images' trait of a MultipleMialsrtkCorrectSliceIntensityInputSpec instance must be a file name, but a value of None <class 'NoneType'> was specified.

Error setting node input:
Node: srtkCorrectSliceIntensity01
input: input_images
results_file: /fetaldata/derivatives/superres-mri/sub-HK01/ses-01/srr_nipype/data_grabber/result_data_grabber.pklz
value: None